# Twitter Scraping


Um Störungen im Betriebsnetz festzustellen, wurden jeweils die Twitter Beiträge des offiziellen SBB Accounts gescrapt. 

In [1]:
#import libraries and modules 
from App.helper.functions import process_twitter_data, get_twitter_data, add_twitter_info
import App.helper.config as config
import pandas as pd
import os

In [2]:
# import data form csv
data = pd.read_csv('App/Daten/data_expanded/combined_data.csv', parse_dates=['BETRIEBSTAG', 'AN_soll', 'AB_soll', 'AN_ist', 'AB_ist'])
data.head()

,BETRIEBSTAG,LINIEN_ID,LINIEN_TEXT,haltestelle_an,AN_soll,AN_ist,AN_diff,haltestelle_ab,AB_soll,AB_ist,AB_diff,weekday,ab_hour,ab_minute
0,2022-04-01,1056,IC61,Basel SBB,2022-04-01 07:32:00,2022-04-01 07:32:49,0.816667,Olten,2022-04-01 07:05:00,2022-04-01 07:07:23,2.383333,4,7,5
1,2022-04-01,1057,IC61,Olten,2022-04-01 05:54:00,2022-04-01 05:54:03,0.050000,Basel SBB,2022-04-01 05:22:00,2022-04-01 05:24:08,2.133333,4,5,22
2,2022-04-01,1058,IC61,Basel SBB,2022-04-01 08:32:00,2022-04-01 08:33:02,1.033333,Olten,2022-04-01 08:05:00,2022-04-01 08:06:37,1.616667,4,8,5
3,2022-04-01,1060,IC6,Basel SBB,2022-04-01 09:32:00,2022-04-01 09:35:07,3.116667,Olten,2022-04-01 09:05:00,2022-04-01 09:12:14,7.233333,4,9,5
4,2022-04-01,1062,IC61,Basel SBB,2022-04-01 10:32:00,2022-04-01 10:31:35,-0.416667,Olten,2022-04-01 10:05:00,2022-04-01 10:06:11,1.183333,4,10,5


In [3]:
# set start and end date
START_DATE = data['BETRIEBSTAG'].min().strftime('%Y-%m-%d')
END_DATE = data['BETRIEBSTAG'].max().strftime('%Y-%m-%d')

# Define twitter query
query = 'from:railinfo_sbb since:' + START_DATE + ' until:' + END_DATE
max_tweets = 15000

# lowercase all items from config.HALTESTELLEN
haltestellen = [x.lower() for x in config.HALTESTELLEN]

tweets_raw = get_twitter_data(query, max_tweets, haltestellen)

# Save twitter data to csv
tweets_raw.to_csv('App/Daten/twitter_data_raw.csv', index=False)


c:\Users\stoec\OneDrive - Universität St.Gallen\Semester 9\Big Data & Data Science\bigdata-fs23\App\helper\functions.py:17: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  if any(keyword in tweet.content.lower() for keyword in haltestellen):
c:\Users\stoec\OneDrive - Universität St.Gallen\Semester 9\Big Data & Data Science\bigdata-fs23\App\helper\functions.py:18: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  attributes_container.append([tweet.date, tweet.likeCount, tweet.sourceLabel, tweet.content])


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500


Stopping after 20 empty pages


In [16]:
# Process twitter data
tweets = pd.read_csv('App/Daten/twitter_data_raw.csv', parse_dates=['Date Created'])
# define all lines
linien = [x.lower() for x in config.LINIEN]

# lowercase all items from config.HALTESTELLEN
haltestellen = [x.lower() for x in config.HALTESTELLEN]

# suppress Warnings
pd.options.mode.chained_assignment = None  # default='warn'
twitter_clean = process_twitter_data(tweets, haltestellen, linien)
# reactivating Warnings
pd.options.mode.chained_assignment = 'warn'  # default='warn'

# Save twitter data to csv
twitter_clean.to_csv('App/Daten/twitter_data_clean.csv', index=False)

# # Delete twitter_data_clean.csv
# try:
#     os.remove('App/Daten/twitter_data_raw.csv')
# except:
#     pass

In [17]:
#Check values
twitter_clean['Einschr_type'].value_counts()


Einschränkungen    361
Endmeldung:        194
Aufgehoben:         42
Einschränkung       19
Unterbruch          10
Verkehr              1
Teilausfall:         1
Name: Einschr_type, dtype: int64

In [18]:
# check values
twitter_clean = twitter_clean[twitter_clean['Einschr_type'] != 'Verkehr']
twitter_clean = twitter_clean[twitter_clean['Einschr_type'] != 'Teilausfall:']
twitter_clean['Einschr_type'].value_counts()         

Einschränkungen    361
Endmeldung:        194
Aufgehoben:         42
Einschränkung       19
Unterbruch          10
Name: Einschr_type, dtype: int64

In [19]:
# check if data_disturbances already exists
if not 'disturbance_overlap' in data.columns:
    data_twitter = add_twitter_info(data, twitter_clean)
    data_twitter.drop_duplicates(inplace=True)
    # Save data to csv
    data_twitter.to_csv('App/Daten/data_expanded/combined_data_twitter.csv', index=False)